In [1]:
import sys
sys.path.append('../')
import corecomposition as cc
from astropy.table import Table

import pyvo as vo
import pandas as pd

# Define the Gaia DR3 source_id you're interested in
gold = Table.read('../data/processed/gold.csv').to_pandas()
gaia_source_ids = gold['wd_source_id']  # Replace with your list of Gaia source_ids

In [2]:
gaia_query = f"""
    SELECT gs.source_id, gs.original_ext_source_id
    FROM gaiadr3.panstarrs1_best_neighbour as gs
    WHERE gs.source_id in {tuple(gaia_source_ids)}
    """

gaia_tap_service = vo.dal.TAPService("https://gea.esac.esa.int/tap-server/tap")
gaia_result = gaia_tap_service.search(gaia_query).to_table()

# Convert Gaia result to a pandas DataFrame
df_gaia = gaia_result.to_pandas()
print("Gaia Data:")
print(df_gaia)

Gaia Data:
             source_id  original_ext_source_id
0  2153552814748001792      177342846587376626
1  2376773024012888832       94930126635901317
2  3970162318385391488      126981690946968363
3  4248619071266347648      114163013511255283
4  5750618956285716480       97251318748580338


In [3]:
# Step 2: Query PANSTARRS for objects near the Gaia positions
# Use a small search radius, e.g., 1 arcsecond (0.00027778 degrees)
panstarrs_query = f"""
SELECT *
FROM dbo.MeanObjectView
WHERE objID in {tuple(gaia_result['original_ext_source_id'])}
AND nDetections >= 2
"""
print(panstarrs_query)
service_panstarrs = vo.dal.TAPService('https://vao.stsci.edu/PS1DR2/tapservice.aspx')
panstarrs_result = service_panstarrs.search(panstarrs_query).to_table()

# Convert PANSTARRS result to a pandas DataFrame
df_panstarrs = panstarrs_result.to_pandas()
print("PANSTARRS Data:")
print(df_panstarrs)


SELECT *
FROM dbo.MeanObjectView
WHERE objID in (177342846587376626, 94930126635901317, 126981690946968363, 114163013511255283, 97251318748580338)
AND nDetections >= 2

PANSTARRS Data:
   zMeanPSFMagErr  zMeanPSFMagStd  zMeanPSFMagNpt  zMeanPSFMagMin  \
0        0.157601        0.291221               2       19.749701   
1        0.035926        0.100038               6       19.758699   
2        0.034080        0.071302               5       19.350800   
3        0.014535        0.039517               7       19.328501   
4        0.027282        0.174210              14       19.806101   

   zMeanPSFMagMax  zMeanKronMag  zMeanKronMagErr  zMeanKronMagStd  \
0       20.064899     19.745300         0.000047      -999.000000   
1       20.022600     19.256399         0.136731         0.364686   
2       19.533199     19.586599         0.058204         0.145735   
3       19.431801     19.538000         0.026786         0.073178   
4       20.508499     20.126801         0.068294      

In [4]:
cc.photometry.panstarrs_photo(gold['wd_source_id'])

source_id,qualityFlag,PS1_g,PS1_r,PS1_i,PS1_z,PS1_y,e_PS1_g,e_PS1_r,e_PS1_i,e_PS1_z,e_PS1_y
,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag
int64,uint8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
2376773024012888832,53,20.0215,19.9431,19.9534,20.0399,19.6576,0.02728,0.016863,0.024493,0.157601,0.065696
5750618956285716480,52,19.8769,19.8533,19.8706,19.9204,19.8858,0.020887,0.014743,0.016776,0.035926,0.051367
4248619071266347648,52,19.163,19.1763,19.2267,19.4824,19.4466,0.008936,0.011638,0.014273,0.03408,0.094009
3970162318385391488,52,19.1436,19.1755,19.2551,19.3702,19.3349,0.011464,0.010883,0.008248,0.014535,0.092098
2153552814748001792,52,19.9119,19.8831,19.9115,20.0249,19.6572,0.014788,0.012435,0.011477,0.027282,0.225687
